In this Jupyter-Notebook we shall take a look at how to extract the early exercise premium of American options. Recall that the Black-Scholes formula for vanilla call and put options are only applicable for European exercise type. First, we reproduce the Black-Scholes formula for European call and put options.

In [3]:
import numpy as np
from scipy.stats import norm


def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)


def BlackScholesPut(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)


For American exercise type, we can no longer use the analytical formula provided by Black and Scholes. However, the binomial tree model we have covered in the earlier sessions can handle American options, through the "backward induction" method. In Session 1 and 2, we have looked at examples pricing American and European vanilla options under the binomial tree model.

In this notebook, we shall first implement the binomial tree model for both American and European vanilla options. Recall in Session 2 Additional Example Question 7's discussion that the $u$ and $d$ parameters in the binomial tree model is related to the Black-Scholes model's sigma through the equation
\begin{equation*}
    u = e^{\sigma \sqrt{\Delta t}}, \hspace{1cm} d = \frac{1}{u} = e^{-\sigma \sqrt{\Delta t}}
\end{equation*}

In [1]:
import array


def american_put(S, K, r, sigma, T, steps):
    R = (1+r)**(T/steps)
    u = np.exp(sigma*np.sqrt(T/steps))
    uu = u*u
    d = 1.0/u
    p_up = (R-d)/(u-d)
    p_down = 1.0-p_up
    prices = array.array('d', (0 for i in range(0, steps+1)))
    prices[0] = S*pow(d, steps)

    for i in range(1, steps+1):
        prices[i] = uu*prices[i-1]

    put_values = array.array('d', (0 for i in range(0, steps+1)))

    for i in range(0, steps+1):
        put_values[i] = max(0.0, (K-prices[i]))

    for step in range(steps-1, -1, -1):
        for i in range(0, step+1):
            put_values[i] = (p_up*put_values[i+1]+p_down*put_values[i])/R
            prices[i] = d*prices[i+1]
            put_values[i] = max(put_values[i], (K-prices[i]))
    return put_values[0]


def european_put(S, K, r, sigma, T, steps):
    R = (1+r)**(T/steps)
    u = np.exp(sigma*np.sqrt(T/steps))
    uu = u*u
    d = 1.0/u
    p_up = (R-d)/(u-d)
    p_down = 1.0-p_up
    prices = array.array('d', (0 for i in range(0, steps+1)))
    prices[0] = S*pow(d, steps)

    for i in range(1, steps+1):
        prices[i] = uu*prices[i-1]

    put_values = array.array('d', (0 for i in range(0, steps+1)))

    for i in range(0, steps+1):
        put_values[i] = max(0.0, (K-prices[i]))

    for step in range(steps-1, -1, -1):
        for i in range(0, step+1):
            put_values[i] = (p_up*put_values[i+1]+p_down*put_values[i])/R
            prices[i] = d*prices[i+1]
    return put_values[0]


def american_call(S, K, r, sigma, T, steps):
    R = (1+r)**(T/steps)
    u = np.exp(sigma*np.sqrt(T/steps))
    d = 1.0/u
    p_up = (R-d)/(u-d)
    p_down = 1.0-p_up

    prices = array.array('d', (0 for i in range(0, steps+1)))

    prices[0] = S*pow(d, steps)
    uu = u*u
    for i in range(1, steps+1):
        prices[i] = uu*prices[i-1]

    call_values = array.array('d', (0 for i in range(0, steps+1)))
    for i in range(0, steps+1):
        call_values[i] = max(0.0, (prices[i]-K))

    for step in range(steps-1, -1, -1):
        for i in range(0, step+1):
            call_values[i] = (p_up*call_values[i+1]+p_down*call_values[i])/R
            prices[i] = d*prices[i+1]
            call_values[i] = max(call_values[i], 1.0*(prices[i]-K))

    return call_values[0]


def european_call(S, K, r, sigma, T, steps):
    R = (1+r)**(T/steps)
    u = np.exp(sigma*np.sqrt(T/steps))
    d = 1.0/u
    p_up = (R-d)/(u-d)
    p_down = 1.0-p_up

    # price of underlying
    prices = array.array('d', (0 for i in range(0, steps+1)))

    # fill in the endnodes
    prices[0] = S*pow(d, steps)
    uu = u*u
    for i in range(1, steps+1):
        prices[i] = uu*prices[i-1]

    call_values = array.array('d', (0 for i in range(0, steps+1)))
    for i in range(0, steps+1):
        call_values[i] = max(0.0, (prices[i]-K))

    for step in range(steps-1, -1, -1):
        for i in range(0, step+1):
            call_values[i] = (p_up*call_values[i+1]+p_down*call_values[i])/R
            prices[i] = d*prices[i+1]

    return call_values[0]



Let us try out our binomial tree implementation on Q2 of Session 2's Additional Examples. Consider a 3-step binomial tree with parameters $S_0=4$, $u=2$, $d=\frac{1}{u}$, $r=0.25$, maturing at time $T=3$, and $K=10$. According to our formula, we have

\begin{equation*}
\sigma = \log(u) \times \frac{1}{\sqrt{\Delta t}} = \log(2) \times \frac{1}{\sqrt{\frac{3}{3}}} = \log(2)
\end{equation*}

In [4]:
S = 4.0
K = 10.0
r = 0.25
T = 3.0
steps = 3
sigma = np.log(2)

In [5]:
print('American Call price: %.4f' % american_call(S, K, r, sigma, T, steps))
print('European Call price: %.4f' % european_call(S, K, r, sigma, T, steps))
print('American Put price: %.4f' % american_put(S, K, r, sigma, T, steps))
print('European Put price: %.4f' % european_put(S, K, r, sigma, T, steps))

American Call price: 1.4080
European Call price: 1.4080
American Put price: 6.0000
European Put price: 2.5280


Next, let us compare the how well do these binomial tree model implementation match the prices from the Black-Scholes formula. To this end, we need to use a significantly larger number of steps in order for the Law of Large Number to apply.

We shall also use more sensible market parameters.

In [6]:
r = 0.05
T = 1.0
K = 5.0
S = 5.0
sigma = 0.15
steps = 2000
print('European Call (binomial tree) price: %.4f' % european_call(S, K, r, sigma, T, steps))
print('European Put (binomial tree) price: %.4f' % european_put(S, K, r, sigma, T, steps))
print('European Call (Black-Scholes) price: %.4f' % BlackScholesCall(S, K, r, sigma, T))
print('European Put (Black-Scholes) price: %.4f' % BlackScholesPut(S, K, r, sigma, T))

European Call (binomial tree) price: 0.4261
European Put (binomial tree) price: 0.1880
European Call (Black-Scholes) price: 0.4296
European Put (Black-Scholes) price: 0.1857
